In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI TabNet

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/tabnet/tabnet_vertex_tutorial.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/tabnet/tabnet_vertex_tutorial.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"> 在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/tabnet/tabnet_vertex_tutorial.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"> 在Vertex AI Workbench中打开
    </a>
  </td>                                                                                               
</table>

## 概览

本教程演示了如何在 Vertex AI 平台上使用 TabNet 算法。

TabNet 结合了两者之间的优点：它具有可解释性（类似于简单的基于树的模型），同时又具有高性能（类似于深度神经网络）。这使得它非常适合零售商、金融和保险行业的应用，比如预测信用评分、欺诈检测和预测等。

TabNet 使用一种称为序列关注的机器学习技术，在模型的每一步中选择要从哪些模型特征推理。这种机制使得能够解释模型如何得出预测，并帮助其学习更准确的模型。由于这种设计，TabNet 不仅比其他神经网络和决策树表现更好，还提供可解释的特征归因。在 Vertex AI 中发布 TabNet 作为一种第一方训练器，意味着您可以轻松利用 TabNet 的架构和可解释性，并用它来对自己的数据训练模型。

了解有关 [Tabular Workflow for TabNet](https://cloud.google.com/vertex-ai/docs/tabular-data/tabular-workflows/tabnet) 的更多信息。

### 目标

在这个笔记本中，您将学习如何在Vertex AI上运行TabNet模型。

本教程使用以下Google Cloud ML服务和资源：

- Vertex AI Training
- Vertex AI Hyperparameter Tuning
- TabNet内置算法
- BigQuery

执行的步骤如下：
1. **设置**: 导入所需的库并设置全局变量。
2. **配置参数**: 设置训练作业的适当参数值。
3. **在Vertex AI上进行训练**: 提交一个使用csv输入的训练作业。
4. **超参数调整**: 运行超参数调整作业。
5. **在Vertex AI上使用BigQuery输入的超参数训练**: 提交一个使用BigQuery输入的训练作业。
6. **清理**: 删除本教程创建的资源。

数据集

本教程使用公共存储桶 `gs://cloud-samples-data/ai-platform-unified/datasets/tabular/` 中的 `petfinder` 数据集，该数据集是从 [PetFinder.my Adoption Prediction](https://www.kaggle.com/c/petfinder-adoption-prediction) 生成的。

### 成本

本教程使用谷歌云的计费组件：

* Vertex AI
* Cloud Storage

了解[Vertex AI价格](https://cloud.google.com/vertex-ai/pricing)和[Cloud Storage价格](https://cloud.google.com/storage/pricing)，并使用[Pricing计算器](https://cloud.google.com/products/calculator/)根据您的预期使用量生成成本估算。

## 安装

安装以下包以执行此笔记本所需的内容。

In [ ]:
! pip3 install --quiet --upgrade tensorflow
! pip3 install --quiet --upgrade google-cloud-aiplatform \
                                 tensorboard-plugin-profile \
                                 google-cloud-pipeline-components
! gcloud components update --quiet

### 仅限Colab：请取消注释以下单元格以重新启动内核。

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

## 开始之前

### 设置您的谷歌云项目

**无论您使用哪种笔记本环境，以下步骤都是必需的。**

1. [选择或创建一个谷歌云项目](https://console.cloud.google.com/cloud-resource-manager)。当您第一次创建帐户时，您将获得 $300 的免费信用额，可用于支付计算/存储成本。

2. [确保您的项目已启用计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

3. [启用以下 API：Vertex AI API，Cloud 资源管理器 API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,cloudresourcemanager.googleapis.com)。

4. 如果您在本地运行这个笔记本，您需要安装 [Cloud SDK](https://cloud.google.com/sdk)。

设定您的项目ID

**如果您不知道您的项目ID**，请尝试以下操作：
* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 查看支持页面：[查找项目ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

地区

您也可以更改 Vertex AI 使用的 `REGION` 变量。了解有关 [Vertex AI 地区](https://cloud.google.com/vertex-ai/docs/general/locations) 的更多信息。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

### 验证您的谷歌云账户

根据您的Jupyter环境，您可能需要手动验证。请按照下面的相关指示操作。

**1. Vertex AI Workbench**
* 不需要进行任何操作，因为您已经验证过了。

**2. 本地JupyterLab实例，取消注释并运行：**

In [ ]:
# ! gcloud auth login

3. 合作，取消注释并运行:

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

查看如何授予您的服务账号Cloud Storage权限，请访问https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples。

### 创建一个云存储桶

创建一个存储桶来存储中间产物，如数据集。

当您使用云 SDK 提交训练作业时，您会将包含训练代码的 Python 包上传到云存储桶中。Vertex AI 将从此包中运行代码。在本教程中，Vertex AI 还会将作业产生的训练模型保存在同一个存储桶中。使用这个模型产物，您可以创建 Vertex AI 模型资源并用于预测。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

只有当您的存储桶尚不存在时：运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l {REGION} -p {PROJECT_ID} {BUCKET_URI}

导入库并定义常量

In [ ]:
import os
import re
import time
from datetime import datetime

from google.cloud import aiplatform

%load_ext tensorboard

In [ ]:
# Please note that if you use csv input, the first column is the label column.

IMPORT_FILE = "petfinder-tabular-classification-tabnet-with-header.csv"
! gsutil cp gs://cloud-samples-data/ai-platform-unified/datasets/tabular/{IMPORT_FILE} {BUCKET_URI}/data/petfinder/train.csv

gcs_source = f"{BUCKET_URI[5:]}/data/petfinder/train.csv"
print("gcs_source: ", gcs_source)

## 配置参数

以下表格显示了所有使用`gcloud ai custom-jobs create`命令创建的Vertex Training作业所共有的参数。查看[官方文档](https://cloud.google.com/sdk/gcloud/reference/ai/custom-jobs/create)以获取所有可能的参数。

| 参数 | 数据类型 | 描述 | 必填 |
|--|--|--|--|
| `display-name` | 字符串 | 作业的名称。 | 是 |
| `worker-pool-spec` | 字符串 | 逗号分隔的参数列表，指定工作池配置（见下文）。 | 是 |
| `region` | 字符串 | 提交作业的区域。 | 否 |

`worker-pool-spec`标志可以多次指定，每个工作池一个。以下表格显示用于指定工作池的参数。

| 参数 | 数据类型 | 描述 | 必填 |
|--|--|--|--|
| `machine-type` | 字符串 | 工作池的机器类型。参阅[官方文档](https://cloud.google.com/vertex-ai/docs/training/configure-compute)以获取支持的机器。 | 是 |
| `replica-count` | 整数 | 工作池中机器的副本数量。 | 否 |
| `container-image-uri` | 字符串 | 每个工作器上运行的Docker镜像。 | 否 |

以下表格显示了TabNet训练作业的参数：

| 参数 | 数据类型 | 描述 | 必填 |
|--|--|--|--|
| `preprocess` | 布尔参数 | 指定启用自动预处理。| 否|
| `job_dir` | 字符串 | 存储模型输出文件的云存储目录。| 是 |
| `input_metadata_path` | 字符串 | 训练数据集的TabNet特定元数据的GCS路径。请参阅上文如何创建元数据。| 否 . |
| `training_data_path` | 字符串 | 存储训练数据的云存储模式。| 是 |
| `validation_data_path` | 字符串 | 存储评估数据的云存储模式。| 否 |
| `test_data_path` | 字符串 | 存储测试数据的云存储模式。| 是 |
| `input_type` | 字符串 | “bigquery”或“csv” - 输入表格数据的类型。如果提到csv，则第一列被视为目标。如果CSV文件有标题，还要传递标志“data_has_header”。如果使用“bigquery”，可以提供训练/验证数据路径，或提供BigQuery项目、数据集和表名称以进行预处理以生成训练和验证数据集。| 否 - 默认为“csv” |
| `model_type` | 字符串 | 学习任务，如分类或回归。| 是 |
| `split_column` | 字符串 | 用于创建训练、验证和测试拆分的列名。列的值（即表['split_column']）应该是“TRAIN”、“VALIDATE”或“TEST”。“TEST”是可选的。仅适用于bigquery输入。| 否 . |
| `train_batch_size` | 整数 | 训练的批量大小。| 否 - 默认为1024。 |
| `eval_split` | 浮点数 | 用于评估数据集的拆分分数，如果未提供`validation_data_path`。| 否 - 默认为0.2 |
| `learning_rate` | 浮点数 | 训练的学习速率。| 否 - 默认为指定优化器的默认学习速率。 |
| `eval_frequency_secs` | 整数 | 评估和检查点将发生的频率。默认为600。| 否 . |
| `num_parallel_reads` | 整数 | 用于读取输入文件的线程数。我们建议将其设置为等于或略少于机器的CPU数以获得最大性能。例如，每个GPU的默认选择是6。| 是 . |
| `data_cache` | 字符串 | 选择将数据缓存到“内存”、“磁盘”或“no_cache”。对于大型数据集，将数据缓存到内存可能会导致内存不足错误，因此我们建议选择“磁盘”。您可以在配置文件中指定磁盘大小（如下所示）。确保请求足够大（例如TB大小）的磁盘来写入数据，对于大规模（B级）数据集。| 否，默认值为“内存” . |
| `bq_project` | 字符串 | BigQuery项目的名称。如果输入类型为bigquery并且使用标志-preprocessing，则需要此项。这是指定训练、验证和测试数据路径的另一种选择。| 否 . |
| `dataset_name` | 字符串 | BigQuery数据集的名称。如果输入类型为bigquery并且使用标志-preprocessing，则需要此项。这是指定训练、验证和测试数据路径的另一种选择。| 否 . |
| `table_name` | 字符串 | BigQuery表的名称。如果输入类型为bigquery并且使用标志-preprocessing，则需要此项。这是指定训练、验证和测试数据路径的另一种选择。| 否 . |
| `loss_function_type` | 字符串 | TabNet中有几种损失函数类型。对于回归：包括mse/mae。对于分类：包括cross_entropy/weighted_cross_entropy/focal_loss | 否 . 如果值为"default"，则我们在回归中使用mse，在分类中使用cross_entropy。|
| `deterministic_data` | 布尔参数 | 从表格数据中读取的确定性。默认值设置为False。将该值设置为True时，实验是确定性的。对于大型数据集的快速训练，我们建议将deterministic_data=False设置，尽管在结果中存在随机性（在大型数据集中变得可以忽略不计）。请注意，仍无法保证分布式训练的确定性，因为map-reduce由于有限精度代数操作的排序而导致随机性。不过，在实践中这在大型数据集上是可以忽略的。对于希望100%确定性的情况，除了设置deterministic_data=True之外，我们建议使用单个GPU进行训练（例如使用MACHINE_TYPE="n1-highmem-8"）。| 否，默认值为False . |
| `stream_inputs` | 布尔参数 | 从GCS流式传输输入数据，而不是在本地下载 - 建议用于快速运行时。| 否 . |
| `large_category_dim` | 整数 | 嵌入的维数 - 如果分类列的不同类别数量大于large_category_thresh，则我们使用large_category_dim维度的嵌入，而不是1维嵌入。默认值为1。我们建议适当增加（例如在大多数情况下增加到~5，在数据集中的类别数量特别大时甚至增加到~10），如果提高准确性是主要目标，而不是计算效率和可解释性。| 否 . |
| `large_category_thresh` | 整数 | 分类列基数的阈值 - 如果分类列的不同类别数量大于large_category_thresh，则我们使用large_category_dim维度的嵌入，而不是1维嵌入。默认值为300。我们建议降低它（例如到~10），如果提高准确性是主要目标，而不是计算效率和可解释性。| 否 . |
| `yeo_johnson_transform` | 布尔参数 | 启用可训练的Yeo-Johnson Power变换（默认值为禁用）。请参阅此链接：https://www.stat.umn.edu/arc/yjpower.pdf 以获取有关Yeo-Johnson Power变换的更多信息。使用我们的实现，变换参数可以与TabNet一起学习，以端到端方式进行训练。| 否 . |
| `apply_log_transform` | 布尔参数 | 如果元数据中包含对数变换统计信息并且该标志为true，则输入特征将被对数转换。使用false不使用变换，true（默认值）使用变换。特别是对于具有偏态数值分布的数据集，对数转换可能非常有用。| 否 . |
| `apply_quantile_transform` | 布尔参数 | 如果元数据中包含分位数统计信息并且该标志为true，则输入特征将被分位数转换。使用false不使用变换，true（默认值）使用变换。目前支持BigQuery输入类型。| 否 . |
| `replace_transformed_features` | 布尔参数 | 如果为true，则如果对特征应用了转换，则该特征将被替换。如果为false（默认选项），则转换后的特征将作为新特征添加到特征列列表中。使用true替换特征为转换，false将转换后的特征追加为新列。| 否 . |
| `target_column` | 字符串 | 标签列的名称。注意，对于分类，标签需要是字符串或整数类型。| 否 . |
| `prediction_raw_inputs` | 布尔参数 | 如果设置了该参数，则模型服务允许我们将特征作为张量字典传递，而不是CSV行。| 否 . |
| `exclude_key` | 布尔参数 | 如果设置了该参数，我们可以排除输入/输出中的一个键。键对于批处理预测过程很有用，它以不可预测的顺序输入和保存输出。键有助于将输出与输入匹配。| 否 . |

In [ ]:
# URI of the TabNet training Docker image.
LEARNER_IMAGE_URI = (
    "us-docker.pkg.dev/vertex-ai-restricted/builtin-algorithm/tab_net_v2"
)

DATASET_NAME = "petfinder"  # Change to your dataset name.

ALGORITHM = "tabnet"
MODEL_TYPE = "classification"
MODEL_NAME = f"{DATASET_NAME}_{ALGORITHM}_{MODEL_TYPE}"

TRAINING_DATA_PATH = f"{BUCKET_URI}/data/{DATASET_NAME}/train.csv"

print("training path: ", TRAINING_DATA_PATH)
print("model name: ", MODEL_NAME)

# 运行TabNet训练

In [ ]:
current_time = datetime.utcnow().strftime('%y%m%d%H%M%S')
OUTPUT_DIR = f"{BUCKET_URI}/{MODEL_NAME}_{current_time}"
print('Output dir: ', OUTPUT_DIR)
learning_job_name = f'tab_net_{DATASET_NAME}_{current_time}'
print('Job name: ', learning_job_name)

CREATION_LOG = !gcloud ai custom-jobs create \
  --region={REGION} \
  --display-name={learning_job_name} \
  --worker-pool-spec=machine-type=n1-standard-8,replica-count=1,container-image-uri={LEARNER_IMAGE_URI} \
  --args=--preprocess \
  --args=--prediction_raw_inputs \
  --args=--exclude_key \
  --args=--data_has_header \
  --args=--training_data_path={TRAINING_DATA_PATH} \
  --args=--job-dir={OUTPUT_DIR} \
  --args=--model_type={MODEL_TYPE} \
  --args=--max_steps=2000 \
  --args=--batch_size=4096 \
  --args=--learning_rate=0.01

print(CREATION_LOG)

工作提交成功后，您可以查看其详细信息和日志：

In [ ]:
JOB_ID = re.search(r"(?<=/customJobs/)\d+", CREATION_LOG[1]).group(0)
print(JOB_ID)

In [ ]:
# View the job's configuration and state.
STATE = "state: JOB_STATE_PENDING"

while STATE not in ["state: JOB_STATE_SUCCEEDED", "state: JOB_STATE_FAILED"]:
    DESCRIPTION = ! gcloud ai custom-jobs describe {JOB_ID} --region={REGION}
    STATE = DESCRIPTION[-2]
    print(STATE)
    time.sleep(60)

# 在 Vertex AI Prediction 上部署

## 导入模型
您的训练作业将在 `gs://<job_dir>/model` 下导出了两个 TF SavedModel。导出的模型可用于在 Vertex AI Prediction 中进行在线或批量预测。首先，导入模型：

In [ ]:
# Imports the model.
current_time = datetime.utcnow().strftime("%y%m%d%H%M%S")
MODEL_TYPE = "classification"

DISPLAY_NAME = f"tabnet_{DATASET_NAME}_{MODEL_TYPE}_{current_time}"  # The display name of the model.
MODEL_NAME = f"tabnet_{MODEL_TYPE}_model"  # Used by the deployment container.
ARTIFACT_URI = f"{OUTPUT_DIR}/model/"

print("Display_name:", DISPLAY_NAME)
print("Model name: ", MODEL_NAME)

aiplatform.init(
    project=PROJECT_ID,
    location=REGION,
    staging_bucket=BUCKET_URI,
)

model = aiplatform.Model.upload(
    display_name=DISPLAY_NAME,
    artifact_uri=ARTIFACT_URI,
    serving_container_image_uri="us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-4:latest",
)
print(model)

部署模型
在导入模型后，您必须将其部署到一个端点，以便获取在线预测。有关此过程的更多信息可以在[官方文档](https://cloud.google.com/vertex-ai/docs/predictions/deploy-model-api)中找到。

创建一个模型终端

In [ ]:
endpoint = aiplatform.Endpoint.create(display_name=DATASET_NAME)

# If you want to use existing endpoint
# endpoint = aiplatform.Endpoint('[location of existing endpoint]')
print("Endpoint: ", endpoint)

将模型部署到端点

In [ ]:
model.deploy(
    endpoint=endpoint,
    machine_type="n1-standard-4",
    traffic_split={"0": 100},
    deployed_model_display_name=DISPLAY_NAME,
)

在线预测

In [ ]:
prediction = endpoint.predict(
    [
        {
            "Type": "Cat",
            "Age": 3,
            "Breed1": "Tabby",
            "Gender": "Male",
            "Color1": "Black",
            "Color2": "White",
            "MaturitySize": "Small",
            "FurLength": "Short",
            "Vaccinated": "No",
            "Sterilized": "No",
            "Health": "Healthy",
            "Fee": 100,
            "PhotoAmt": 2,
        }
    ]
)

print(prediction)

超参数调优
在成功训练模型、部署模型并调用模型进行预测后，您可能希望优化训练过程中使用的超参数，以提高模型的准确性和性能。请查阅 Vertex AI 文档，了解超参数调整的概述以及如何在 Vertex 训练作业中使用它。

例如，以下命令将运行一个包含4个试验的 Vertex AI 超参数调整作业，试图最大化验证AUC指标。它优化的超参数是最大步数和学习率。

In [ ]:
DATASET_NAME = "petfinder"  # Change to your dataset name.
current_time = datetime.utcnow().strftime('%y%m%d%H%M%S')
OUTPUT_DIR = f"{BUCKET_URI[5:]}/hptuning/{MODEL_NAME}_{current_time}/"
print('Output dir: ', OUTPUT_DIR)
learning_job_name = f'tab_net_{DATASET_NAME}_{current_time}'
print('Job name: ', learning_job_name)

config = f"""studySpec:
  metrics:
  - metricId: auc
    goal: MAXIMIZE
  parameters:
  - parameterId: max_steps
    integerValueSpec:
      minValue: 2000
      maxValue: 3000
  - parameterId: learning_rate
    doubleValueSpec:
      minValue: 0.0000001
      maxValue: 0.1
trialJobSpec:
  workerPoolSpecs:
  - machineSpec:
      machineType: n1-highmem-8
      acceleratorType: NVIDIA_TESLA_V100
      acceleratorCount: 1
    replicaCount: 1
    diskSpec:
      bootDiskType: pd-ssd
      bootDiskSizeGb: 100
    containerSpec:
      imageUri: {LEARNER_IMAGE_URI}
      args:
      - --preprocess 
      - --prediction_raw_inputs
      - --exclude_key
      - --data_has_header
      - --training_data_path={TRAINING_DATA_PATH}
      - --job-dir={OUTPUT_DIR}
      - --batch_size=1028
      - --model_type=classification
"""

!echo $'{config}' > ./config.yaml

MAX_TRIAL_COUNT=4
PARALLEL_TRIAL_COUNT=2

!gcloud ai hp-tuning-jobs create \
  --config=config.yaml \
  --max-trial-count={MAX_TRIAL_COUNT} \
  --parallel-trial-count={PARALLEL_TRIAL_COUNT} \
  --region=$REGION \
  --display-name=$learning_job_name

# 使用Big Query（BQ）输入进行超参数调整
您需要使用上述csv文件创建BQ数据集。

在您的项目中创建BQ数据集

In [ ]:
! bq --location={REGION} mk --dataset {PROJECT_ID}:{DATASET_NAME}

使用csv文件创建BQ表。

In [ ]:
!bq --location={REGION} load --source_format=CSV --autodetect {PROJECT_ID}:{DATASET_NAME}.train {BUCKET_URI}/data/petfinder/train.csv

使用BQ输入运行hp-tuning。

In [ ]:
current_time = datetime.utcnow().strftime('%y%m%d%H%M%S')

BQ_PROJECT=f"{PROJECT_ID}"
DATASET_NAME = "petfinder"  # Change to your dataset name.
TABLE_NAME="train"
OUTPUT_DIR = f"{BUCKET_URI}/hptuning/{MODEL_NAME}_{current_time}/"
print('Output dir: ', OUTPUT_DIR)
learning_job_name = f'tab_net_{DATASET_NAME}_{current_time}'
print('Job name: ', learning_job_name)

config = f"""studySpec:
  metrics:
  - metricId: auc
    goal: MAXIMIZE
  parameters:
  - parameterId: max_steps
    integerValueSpec:
      minValue: 2000
      maxValue: 3000
  - parameterId: learning_rate
    doubleValueSpec:
      minValue: 0.0000001
      maxValue: 0.1
trialJobSpec:
  workerPoolSpecs:
  - machineSpec:
      machineType: n1-highmem-8
      acceleratorType: NVIDIA_TESLA_V100
      acceleratorCount: 1
    replicaCount: 1
    diskSpec:
      bootDiskType: pd-ssd
      bootDiskSizeGb: 200
    containerSpec:
      imageUri: {LEARNER_IMAGE_URI}
      args:
      - --eval_frequency_secs=10800
      - --input_type=bigquery
      - --preprocess
      - --prediction_raw_inputs
      - --exclude_key
      - --model_type=classification
      - --stream_inputs
      - --bq_project={BQ_PROJECT}
      - --dataset_name={DATASET_NAME}
      - --table_name={TABLE_NAME}
      - --target_column=Adopted
      - --num_parallel_reads=2
      - --optimizer_type=adam
      - --data_cache=disk
      - --deterministic_data=False
      - --loss_function_type=weighted_cross_entropy
      - --replace_transformed_features=True
      - --apply_quantile_transform=True
      - --apply_log_transform=True
      - --batch_size=32768
      - --job-dir={OUTPUT_DIR}
"""

!echo $'{config}' > ./hptuning-config.yaml

MAX_TRIAL_COUNT=2
PARALLEL_TRIAL_COUNT=2

!gcloud ai hp-tuning-jobs create \
  --config=hptuning-config.yaml \
  --max-trial-count={MAX_TRIAL_COUNT} \
  --parallel-trial-count={PARALLEL_TRIAL_COUNT} \
  --region=$REGION \
  --display-name=$learning_job_name

清理工作

要清理此项目中使用的所有谷歌云资源，您可以删除用于本教程的[谷歌云项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的各个资源。

In [ ]:
# Delete Cloud Storage objects that were created
! gsutil -m rm -r $JOB_DIR

# Delete BQ table
! bq rm -f {PROJECT_ID}:{DATASET_NAME}.train

# Delete endpoint resource
endpoint.delete(force=True)

# Delete model resource
model.delete()

if os.getenv("IS_TESTING"):
    ! gsutil -m rm -r $BUCKET_URI